## I. 패키지 밎 자료 불러오기

In [ ]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient

     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 87 kB 4.5 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99964 sha256=8a7e229fb4406e63a61312b8f2dea20c2984f69e2ff61d2d33494fe41c9bbabd
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [ ]:
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 135 kB 7.7 MB/s 
     |████████████████████████████████| 130 kB 54.3 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line
import plotly.offline as plyo
import cufflinks
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
db = pymysql.connect(host, charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM cj'
cursor.execute(sql)
result = cursor.fetchall()
DF = pd.DataFrame(result)


OperationalError: ignored

In [ ]:
DF

,index,date,coin_name,s,s.1,lang
0,8,2021-11-22,ada,0,9,kor
1,9,2021-11-22,ada,1,7,kor
2,10,2021-11-22,bch,2,9,kor
3,11,2021-11-22,bch,0,7,kor
4,12,2021-11-22,bch,1,6,kor
...,...,...,...,...,...,...
221,14,2021-11-24,omg,1,1,kor
222,15,2021-11-24,snt,2,1,kor
223,16,2021-11-24,xrp,0,35,kor
224,17,2021-11-24,xrp,2,21,kor


In [ ]:
DF.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_db_2.csv')

## I. 일단 긍정/중립/부정 3개의 데이터 프래임으로 모으기
- 없을시 0으로 해야 함으로 데이터 프레임을 먼저 만들고 거기에 합치기
- s.1은 갯수

In [ ]:
date = ['2021-11-22','2021-11-23','2021-11-24']
# lang = ['kor','eng']
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
s = [0,1,2]

### 1) 새로운 데이터 프레임을 만들어야한다.

In [ ]:
# 새로운 데이터 프레임 만들기
df = pd.DataFrame(columns = ['date', 'crypto_name', 'positive_E','neutrual_E','negative_E','positive_K','neutrual_K','negative_K'])
for i in date:
  for j in crpyto_name:
    df = df.append(pd.DataFrame([[i, j]], columns = ['date','crypto_name']), ignore_index=True)
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2021-11-22,ada,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-11-22,bch,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-11-22,btc,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-11-22,doge,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-22,eos,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-11-22,eth,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-11-22,omg,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-11-22,snt,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-11-22,xrp,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-11-23,ada,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # 연습 빈공간 지정해서 넣기
# for i in range(len(DF)):
#   if (DF['date'][i] == '2021-11-22') and (DF['coin_name'][i] == 'ada') and(DF['lang'][i] == 'kor'):
#     if (DF['s'][i] == 0):
#       print(DF['s.1'][i])
#       a = df.index[(df['date'] == '2021-11-22')&(df['crypto_name'] == 'ada')].tolist()
#       df['positive_K'][a] = DF['s.1'][i]
#     else:
#       print('없음')

9
없음


### 2) 확장 - 모든 코인 모든 날짜

In [ ]:
# for j in date:

#   for i in range(len(DF)):


#     if (DF['date'][i] == j) and (DF['coin_name'][i] == 'ada') and (DF['lang'][i] == 'kor'):
#       if (DF['s'][i] == 0):
#         a = df.index[(df['date'] == j)&(df['crypto_name'] == 'ada')&(DF['lang'][i] == 'kor')].tolist()
#         df['positive_K'][a] = DF['s.1'][i]

In [ ]:
for k in crpyto_name:

  for j in date:

    for i in range(len(DF)):


      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['positive_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['positive_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['neutrual_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['neutrual_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['negative_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['negative_E'][a] = DF['s.1'][i]    

In [ ]:
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2021-11-22,ada,741,1,1692,9,7,NaN
1,2021-11-22,bch,4,NaN,13,7,6,9
2,2021-11-22,btc,143,4,296,3,6,NaN
3,2021-11-22,doge,79,3,175,18,2,8
4,2021-11-22,eos,4,NaN,12,24,17,13
5,2021-11-22,eth,296,1,677,2,NaN,NaN
6,2021-11-22,omg,NaN,NaN,4,2,NaN,NaN
7,2021-11-22,snt,14,NaN,2,3,NaN,4
8,2021-11-22,xrp,40,NaN,112,18,2,19
9,2021-11-23,ada,4727,8,22245,3,2,2


In [ ]:
df = df.fillna(0)

In [ ]:
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2021-11-22,ada,741,1,1692,9,7,0
1,2021-11-22,bch,4,0,13,7,6,9
2,2021-11-22,btc,143,4,296,3,6,0
3,2021-11-22,doge,79,3,175,18,2,8
4,2021-11-22,eos,4,0,12,24,17,13
5,2021-11-22,eth,296,1,677,2,0,0
6,2021-11-22,omg,0,0,4,2,0,0
7,2021-11-22,snt,14,0,2,3,0,4
8,2021-11-22,xrp,40,0,112,18,2,19
9,2021-11-23,ada,4727,8,22245,3,2,2


## --------- 전처리 작업 완료

In [ ]:
# 저장포인트
df.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_graphing.csv')

In [ ]:
# 저장포인트
ADA.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/ADA.csv')
BCH.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/BCH.csv')
BTC.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/BTC.csv')
DOGE.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/DOGE.csv')
EOS.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/EOS.csv')
ETH.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/ETH.csv')
OMG.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/OMG.csv')
SNT.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/SNT.csv')
XRP.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/XRP.csv')

In [ ]:
DF = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_graphing.csv')

In [ ]:
ADA = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/ADA.csv')
BCH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/BCH.csv')
BTC = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/BTC.csv')
DOGE = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/DOGE.csv')
EOS = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/EOS.csv')
ETH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/ETH.csv')
OMG = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/OMG.csv')
SNT = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/SNT.csv')
XRP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_price/XRP.csv')

## -------- 코랩에서 저장 포인트

## II. 지수화
- 일단 긍정의 전체비율만을 가지고 한다. (대부분의 경우 상승에 관심 있고 롱 포지션을 가지기때문에)
- 영어가 대부분이고 영어는 거의 중립이 없어서 가능할지도
- 나중에 부정의 비율도 고려한다.


In [ ]:
date = ['2021-11-22','2021-11-23','2021-11-24']
# lang = ['kor','eng']
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
s = [0,1,2]

### 1) 긍정비율

In [ ]:
DF['positive_ratio_E'] = DF['positive_E']/(DF['positive_E']+DF['neutrual_E']+DF['negative_E'])
DF['positive_ratio_K'] = DF['positive_K']/(DF['positive_K']+DF['neutrual_K']+DF['negative_K'])

In [ ]:
DF

,Unnamed: 0,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,0,2021-11-22,ada,741,1,1692,9,7,0,0.304437,0.562500
1,1,2021-11-22,bch,4,0,13,7,6,9,0.235294,0.318182
2,2,2021-11-22,btc,143,4,296,3,6,0,0.322799,0.333333
3,3,2021-11-22,doge,79,3,175,18,2,8,0.307393,0.642857
4,4,2021-11-22,eos,4,0,12,24,17,13,0.250000,0.444444
5,5,2021-11-22,eth,296,1,677,2,0,0,0.303901,1.000000
6,6,2021-11-22,omg,0,0,4,2,0,0,0.000000,1.000000
7,7,2021-11-22,snt,14,0,2,3,0,4,0.875000,0.428571
8,8,2021-11-22,xrp,40,0,112,18,2,19,0.263158,0.461538
9,9,2021-11-23,ada,4727,8,22245,3,2,2,0.175204,0.428571


## III. 암호화폐별 추출 밎 가격테이블 연결

### 1) 가격데이터, 감정데이터 불러오기 각각 분리

In [ ]:
crypto_name_p = ['ADA','BTC','BCH','DOGE','EOS','ETH','OMG','SNT','XRP']

for i in crypto_name_p:
    db = pymysql.connect(, charset='utf8')
    cursor = db.cursor(pymysql.cursors.DictCursor)
    sql = 'SELECT * FROM {0}'.format(i)
    cursor.execute(sql)
    result = cursor.fetchall()
    globals()[f'{i}'] = pd.DataFrame(result)



crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]

for j1,j2,k in zip(crypto_p,crypto_p,crypto_name_p):

  X = []

  for i in range(len(j1)):
    date_time_obj = datetime.strftime(j2['index'][i], '%Y-%m-%d')
    a = date_time_obj
    b = j2['close'][i]
    # c = j2['value'][i]
    d = j2['volume'][i]
    e = 0
    f = 0
    g = 0
    h = 0
    i = 0
    j = 0

    x = [a,b,d,e,f,g,h,i,j]
    X.append(x)
  globals()['{0}'.format(k)] = pd.DataFrame(X)
  globals()['{0}'.format(k)].columns = ['date','close_KRW','volume','positive_E','neutrual_E','negative_E',	'positive_K',	'neutrual_K',	'negative_K']

In [ ]:
ADA

,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2017-10-06,21.3,3.330000e-02,0,0,0,0,0,0
1,2017-10-07,23.7,5.200000e-02,0,0,0,0,0,0
2,2017-10-08,23.2,5.200000e-02,0,0,0,0,0,0
3,2017-10-09,25.3,5.190000e-02,0,0,0,0,0,0
4,2017-10-10,24.7,5.140000e-02,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1504,2021-11-21,2295.0,2.164858e+07,0,0,0,0,0,0
1505,2021-11-22,2230.0,2.351576e+07,0,0,0,0,0,0
1506,2021-11-23,2190.0,3.456718e+07,0,0,0,0,0,0
1507,2021-11-24,2095.0,1.026309e+08,0,0,0,0,0,0


### 2) 가격데이터에 감정데이터 붙여 넣기
- 가격데이터가 더많이 있어서

In [ ]:
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']

for k,l in zip(crypto_p,crpyto_name):

  for j in date:

      a = DF.index[(DF['date'] == j)&(DF['crypto_name'] == l)].tolist()
      b = k.index[(k['date'] == j)].tolist()
      k['positive_E'][b] = DF['positive_E'][a]
      k['neutrual_E'][b] = DF['neutrual_E'][a]
      k['negative_E'][b] = DF['negative_E'][a]
      k['positive_K'][b] = DF['positive_K'][a]
      k['neutrual_K'][b] = DF['neutrual_K'][a]
      k['negative_K'][b] = DF['negative_K'][a]

## ------------ 중간저장

In [ ]:
# 중간저장
ADA.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ADA.csv')
BCH.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BCH.csv')
BTC.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BTC.csv')
DOGE.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/DOGE.csv')
EOS.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/EOS.csv')
ETH.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ETH.csv')
OMG.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/OMG.csv')
SNT.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/SNT.csv')
XRP.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP.csv')

In [ ]:
# 불러오기
ADA = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ADA.csv')
BCH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BCH.csv')
BTC = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BTC.csv')
DOGE = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/DOGE.csv')
EOS = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/EOS.csv')
ETH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ETH.csv')
OMG = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/OMG.csv')
SNT = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/SNT.csv')
XRP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP.csv')

## IV. 필요데이터만 남기기
-현재는 긍정비율만

## V. 시각화
- 문제점 스케일이 다르다.

In [ ]:
ADA

,Unnamed: 0,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,0,2017-10-06,21.3,3.330000e-02,0,0,0,0,0,0
1,1,2017-10-07,23.7,5.200000e-02,0,0,0,0,0,0
2,2,2017-10-08,23.2,5.200000e-02,0,0,0,0,0,0
3,3,2017-10-09,25.3,5.190000e-02,0,0,0,0,0,0
4,4,2017-10-10,24.7,5.140000e-02,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1504,1504,2021-11-21,2295.0,2.164858e+07,0,0,0,0,0,0
1505,1505,2021-11-22,2230.0,2.351576e+07,741,1,1692,9,7,0
1506,1506,2021-11-23,2190.0,3.456718e+07,4727,8,22245,3,2,2
1507,1507,2021-11-24,2095.0,1.026309e+08,62807,261,154246,12,4,9


In [ ]:
ADA2 = ADA.replace(0,'')

In [ ]:
ADA

In [ ]:
ADA1 = ADA.astype(str)

In [ ]:
fig = px.line(ADA, x="date", y="close_KRW" )
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=ADA2['date'],y=ADA2['close_KRW'],name='KRW',yaxis='y'))
fig.add_trace(go.Scatter(x=ADA2['date'],y=ADA2['positive_E'],name='positive_E',yaxis='y2'))
# fig.add_trace(go.Scatter(x=ADA2['date'],y=ADA2['negative_E'],name='negative_E',yaxis='y3'))

fig.update_traces(
    hoverinfo="name+x+text",
    line={"width": 0.5},
    marker={"size": 0.6},
    mode="lines+markers",
    showlegend=False
)


fig.update_layout(
    xaxis=dict(
        autorange=True,
        range=["2021-10-31", "2021-11-25"],
        rangeslider=dict(
            autorange=True,
            range=["2021-10-31", "2021-11-25"]
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0.3, 1],
        linecolor="#673ab7",
        mirror=True,
        range=[-60.0858369099, 1000],
        showline=True,
        side="left",
        tickfont={"color": "#673ab7"},
        tickmode="auto",
        ticks="",
        titlefont={"color": "#673ab7"},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.1],
        linecolor="#E91E63",
        mirror=True,
        range=[29.3787777032, 100000],
        showline=True,
        side="right",
        tickfont={"color": "#E91E63"},
        tickmode="auto",
        ticks="",
        titlefont={"color": "#E91E63"},
        type="linear",
        zeroline=False
    )
)

fig.update_layout(
    dragmode="zoom",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100
    ),
)


In [ ]:
# Create figure
fig = go.Figure()

# Add traces
# 이름 설정
fig.add_trace(go.Scatter(
    x=["2013-01-15", "2013-01-29", "2013-02-26", "2013-04-19", "2013-07-02",
       "2013-08-27",
       "2013-10-22", "2014-01-20", "2014-05-05", "2014-07-01", "2015-02-09",
       "2015-04-13",
       "2015-05-13", "2015-06-08", "2015-08-05", "2016-02-25"],
    y=["8", "3", "2", "10", "5", "5", "6", "8", "3", "3", "7", "5", "10", "10", "9",
       "14"],
    name="var0",
    text=["8", "3", "2", "10", "5", "5", "6", "8", "3", "3", "7", "5", "10", "10", "9",
          "14"],
    yaxis="y",
))

fig.add_trace(go.Scatter(
    x=["2015-04-13", "2015-05-13", "2015-06-08", "2015-08-05", "2016-02-25"],
    y=["53.0", "69.0", "89.0", "41.0", "41.0"],
    name="var1",
    text=["53.0", "69.0", "89.0", "41.0", "41.0"],
    yaxis="y2",
))

fig.add_trace(go.Scatter(
    x=["2013-01-29", "2013-02-26", "2013-04-19", "2013-07-02", "2013-08-27",
       "2013-10-22",
       "2014-01-20", "2014-04-09", "2014-05-05", "2014-07-01", "2014-09-30",
       "2015-02-09",
       "2015-04-13", "2015-06-08", "2016-02-25"],
    y=["9.6", "4.6", "2.7", "8.3", "18", "7.3", "3", "7.5", "1.0", "0.5", "2.8", "9.2",
       "13", "5.8", "6.9"],
    name="var2",
    text=["9.6", "4.6", "2.7", "8.3", "18", "7.3", "3", "7.5", "1.0", "0.5", "2.8",
          "9.2",
          "13", "5.8", "6.9"],
    yaxis="y3",
))

fig.add_trace(go.Scatter(
    x=["2013-01-29", "2013-02-26", "2013-04-19", "2013-07-02", "2013-08-27",
       "2013-10-22",
       "2014-01-20", "2014-04-09", "2014-05-05", "2014-07-01", "2014-09-30",
       "2015-02-09",
       "2015-04-13", "2015-06-08", "2016-02-25"],
    y=["6.9", "7.5", "7.3", "7.3", "6.9", "7.1", "8", "7.8", "7.4", "7.9", "7.9", "7.6",
       "7.2", "7.2", "8.0"],
    name="var3",
    text=["6.9", "7.5", "7.3", "7.3", "6.9", "7.1", "8", "7.8", "7.4", "7.9", "7.9",
          "7.6",
          "7.2", "7.2", "8.0"],
    yaxis="y4",
))

fig.add_trace(go.Scatter(
    x=["2013-02-26", "2013-07-02", "2013-09-26", "2013-10-22", "2013-12-04",
       "2014-01-02",
       "2014-01-20", "2014-05-05", "2014-07-01", "2015-02-09", "2015-05-05"],
    y=["290", "1078", "263", "407", "660", "740", "33", "374", "95", "734", "3000"],
    name="var4",
    text=["290", "1078", "263", "407", "660", "740", "33", "374", "95", "734", "3000"],
    yaxis="y5",
))

# style all the traces
# 선의 굵기등
fig.update_traces(
    hoverinfo="name+x+text",
    line={"width": 0.5},
    marker={"size": 8},
    mode="lines+markers",
    showlegend=False
)

# Add annotations
fig.update_layout(
    annotations=[
        dict(
            x="2013-06-01",
            y=0,
            arrowcolor="rgba(63, 81, 181, 0.2)",
            arrowsize=0.3,
            ax=0,
            ay=30,
            text="state1",
            xref="x",
            yanchor="bottom",
            yref="y"
        ),
        dict(
            x="2014-09-13",
            y=0,
            arrowcolor="rgba(76, 175, 80, 0.1)",
            arrowsize=0.3,
            ax=0,
            ay=30,
            text="state2",
            xref="x",
            yanchor="bottom",
            yref="y"
        )
    ],
)

# # Add shapes
# # 구간 설정(차트의 배경의 색 구분)
# fig.update_layout(
#     shapes=[
#         dict(
#             fillcolor="rgba(63, 81, 181, 0.2)",
#             line={"width": 0},
#             type="rect",
#             x0="2013-01-15",
#             x1="2013-10-17",
#             xref="x",
#             y0=0,
#             y1=0.95,
#             yref="paper"
#         ),
#         dict(
#             fillcolor="rgba(76, 175, 80, 0.1)",
#             line={"width": 0},
#             type="rect",
#             x0="2013-10-22",
#             x1="2015-08-05",
#             xref="x",
#             y0=0,
#             y1=0.95,
#             yref="paper"
#         )
#     ]
# )

# Update axes
# xaxis 는 아래 부분
# yaxis 는 y 축의 가장 아래 부분
# side = 는 y축 어디에 보이게 할건지
# domain 는 0,1사이 값으로 얼마의 비중으로 보일지
fig.update_layout(
    xaxis=dict(
        autorange=True,
        range=["2012-10-31 18:36:37.3129", "2016-05-10 05:23:22.6871"],
        rangeslider=dict(
            autorange=True,
            range=["2012-10-31 18:36:37.3129", "2016-05-10 05:23:22.6871"]
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.1],
        linecolor="#673ab7",
        mirror=True,
        range=[-60.0858369099, 28.4406294707],
        showline=True,
        side="left",
        tickfont={"color": "#673ab7"},
        tickmode="auto",
        ticks="",
        titlefont={"color": "#673ab7"},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0.2, 0.4],
        linecolor="#E91E63",
        mirror=True,
        range=[29.3787777032, 100.621222297],
        showline=True,
        side="right",
        tickfont={"color": "#E91E63"},
        tickmode="auto",
        ticks="",
        titlefont={"color": "#E91E63"},
        type="linear",
        zeroline=False
    ),
    yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.4, 0.6],
        linecolor="#795548",
        mirror=True,
        range=[-3.73690396239, 22.2369039624],
        showline=True,
        side="right",
        tickfont={"color": "#795548"},
        tickmode="auto",
        ticks="",
        title="mg/L",
        titlefont={"color": "#795548"},
        type="linear",
        zeroline=False
    ),
    yaxis4=dict(
        anchor="x",
        autorange=True,
        domain=[0.6, 0.8],
        linecolor="#607d8b",
        mirror=True,
        range=[6.63368032236, 8.26631967764],
        showline=True,
        side="right",
        tickfont={"color": "#607d8b"},
        tickmode="auto",
        ticks="",
        title="mmol/L",
        titlefont={"color": "#607d8b"},
        type="linear",
        zeroline=False
    ),
    yaxis5=dict(
        anchor="x",
        autorange=True,
        domain=[0.8, 1],
        linecolor="#2196F3",
        mirror=True,
        range=[-685.336803224, 3718.33680322],
        showline=True,
        side="right",
        tickfont={"color": "#2196F3"},
        tickmode="auto",
        ticks="",
        title="mg/Kg",
        titlefont={"color": "#2196F3"},
        type="linear",
        zeroline=False
    )
)

# Update layout
fig.update_layout(
    dragmode="zoom",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100
    ),
)

fig.show()